In [48]:
from datasets import Dataset, DatasetDict
import pandas as pd
from transformers import (AutoModelForSequenceClassification, Trainer, TrainingArguments)
import evaluate
import torch.nn as nn
import torch
import numpy as np
from utils import PROD_TOK, AUX_TOK

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load and save data

## Split data

In [4]:
#Load data
df = pd.read_hdf('human_data.h5', key='df')
df.head()

,Input_seq,Output,Type
0,"[5, 7, 4, 4, 16, 5, 5, 6, 5, 2, 17, 0, 13, 6, ...",[5],Piezas
1,"[1, 6, 8, 4, 1, 9, 11, 7, 2, 5, 8, 10, 1, 12, ...",[5],Productos
2,"[2, 1]",[3],Productos
3,"[9, 5, 6, 9, 11, 1, 2, 6, 8, 12, 0, 6, 7, 10, ...",[7],Productos


In [5]:
#We divide the dataset into products and pieces
df_prod = df.loc[df.Type=='Productos']
df_piez = df.loc[df.Type=='Piezas']

In [6]:
#Save splitted datasets for later use
df_prod.to_hdf('products_human.h5', key='df_prod', index=False)
df_piez.to_hdf('pieces_human.h5', key='df_piez', index=False)

/tmp/ipykernel_1338/2260775344.py:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['Input_seq', 'Output', 'Type'], dtype='object')]

  df_prod.to_hdf('products_human.h5', key='df_prod', index=False)
/tmp/ipykernel_1338/2260775344.py:3: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['Input_seq', 'Output', 'Type'], dtype='object')]

  df_piez.to_hdf('pieces_human.h5', key='df_piez', index=False)


## Load data

In [7]:
#Load splitted datasets
df_prod = pd.read_hdf('products_human.h5', key='df_prod')
df_piez = pd.read_hdf('pieces_human.h5', key='df_piez')

# Load and build model

In [50]:
#We define a custom model as our reward model
class RewardModel (torch.nn.Module):
    def __init__(self, model_name, hidden_size,*model_args, **kwargs):
        super().__init__()
        self.base = AutoModelForSequenceClassification.from_pretrained(model_name,*model_args, **kwargs)
        #Last layer must be changed for it to be a regression problem
        self.base.classifier =  torch.nn.Linear(in_features=hidden_size, out_features=1, bias=True)
        self = self.base
        
    #Our custom model should take as input a pair of right/wrong answers with a fixed sequence
    def forward(self, input_ids, attention_mask=None,
                token_type_ids=None, train:bool=False):
        #Whether the model is being trained or tested is important when inferencing
        if train:
            #The input_ids are structured in a way in which the first chunk corresponds to right examples and the second one to wrong ones
            print(input_ids)
            input_right, input_wrong  = input_ids[0][0], input_ids[0][1]
            out_right = self.base(input_ids=input_right, attention_mask=attention_mask, token_type_ids=token_type_ids)
            out_wrong = self.base(input_ids=input_wrong, attention_mask=attention_mask, token_type_ids=token_type_ids)
            return [out_right, out_wrong]
        else:
            return self.base(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

In [51]:
#We initialize our custom reward model
rm = RewardModel("VCNC/Auto-CNC", hidden_size=768).to(device)

In [43]:
#This is an example of how to inference the model with batches of data
input = torch.tensor([[[[2, 2, 1, 3, 4, 1], [3, 2, 1, 3, 4, 4]], [[2, 2, 1, 3, 4, 3], [3, 2, 1, 3, 4, 1]]]]).to(device)
token_type = torch.tensor([[0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1]]).to(device)
rm(input, token_type, train=True)

tensor([[[[2, 2, 1, 3, 4, 1],
          [3, 2, 1, 3, 4, 4]],

         [[2, 2, 1, 3, 4, 3],
          [3, 2, 1, 3, 4, 1]]]], device='cuda:0')


[SequenceClassifierOutput(loss=None, logits=tensor([[1.2936],
         [0.4296]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None),
 SequenceClassifierOutput(loss=None, logits=tensor([[0.8012],
         [0.6409]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)]

In [11]:
#We can see that the results do not vary, indicating that our functions is correct
input = torch.tensor([[2, 2, 1, 3, 4, 1], [3, 2, 1, 3, 4, 4]]).to(device)
token_type = torch.tensor([[0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1]]).to(device)
rm(input, token_type, train=False)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.2649],
        [-0.2563]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

# Preprocess data

In [12]:
#In this case we will use the product dataset
data = df_prod.to_numpy()
keys = np.unique(np.array(list(PROD_TOK.values()))) #Get unique tokens only
dataset = np.zeros((len(data)*(len(keys)-1), 3), dtype=object) #The number of pairs is the number of examples at the beginning times 1 minus keys

#Iterate through the dataset to generate sets (correct, wrong, input)
for i in range(len(data)):
    internal_cont = 0
    for j in range(len(keys)):
        if j != data[i, 1][0]:
            dataset[i*(len(keys)-1)+internal_cont, 0] = data[i, 1][0]
            dataset[i*(len(keys)-1)+internal_cont, 1] = j
            dataset[i*(len(keys)-1)+internal_cont, 2] = data[i, 0]
            internal_cont += 1
dataset

array([[5, 0,
        list([1, 6, 8, 4, 1, 9, 11, 7, 2, 5, 8, 10, 1, 12, 7, 12, 6, 6, 11, 8, 3, 12, 7, 9, 11, 9, 5])],
       [5, 1,
        list([1, 6, 8, 4, 1, 9, 11, 7, 2, 5, 8, 10, 1, 12, 7, 12, 6, 6, 11, 8, 3, 12, 7, 9, 11, 9, 5])],
       [5, 2,
        list([1, 6, 8, 4, 1, 9, 11, 7, 2, 5, 8, 10, 1, 12, 7, 12, 6, 6, 11, 8, 3, 12, 7, 9, 11, 9, 5])],
       [5, 3,
        list([1, 6, 8, 4, 1, 9, 11, 7, 2, 5, 8, 10, 1, 12, 7, 12, 6, 6, 11, 8, 3, 12, 7, 9, 11, 9, 5])],
       [5, 4,
        list([1, 6, 8, 4, 1, 9, 11, 7, 2, 5, 8, 10, 1, 12, 7, 12, 6, 6, 11, 8, 3, 12, 7, 9, 11, 9, 5])],
       [5, 6,
        list([1, 6, 8, 4, 1, 9, 11, 7, 2, 5, 8, 10, 1, 12, 7, 12, 6, 6, 11, 8, 3, 12, 7, 9, 11, 9, 5])],
       [5, 7,
        list([1, 6, 8, 4, 1, 9, 11, 7, 2, 5, 8, 10, 1, 12, 7, 12, 6, 6, 11, 8, 3, 12, 7, 9, 11, 9, 5])],
       [5, 8,
        list([1, 6, 8, 4, 1, 9, 11, 7, 2, 5, 8, 10, 1, 12, 7, 12, 6, 6, 11, 8, 3, 12, 7, 9, 11, 9, 5])],
       [5, 9,
        list([1, 6, 8, 4, 1, 9, 11

## Data formatting and train/test splitting

In [17]:
MAX_LEN = max([len(i) for i in dataset[:, 2]])
input_ids = []
attention_mask = []
token_type_ids = []
train = []
for i in range(len(dataset)):
    pad_len = MAX_LEN-len(dataset[:, 2][i])
    right_seq = dataset[:, 2][i]+[dataset[i, 0]]+[30]*pad_len
    wrong_seq = dataset[:, 2][i]+[dataset[i, 1]]+[30]*pad_len
    input_ids.append([right_seq, wrong_seq]) #The input_ids is a concatenation of the right and wrong example
    attention_mask.append([1]*(len(dataset[:, 2][i])+1)+[0]*pad_len)
    token_type_ids.append([0]*len(dataset[:, 2][i])+[1]*(pad_len+1)) #The +1 is because the output selected by the human changes the token type
    train.append(True) #In RLHF every example is given as a training example REVISAR!!! 
input_ids[0], attention_mask[0], token_type_ids[0], train[0] #Get first element

([[1,
   6,
   8,
   4,
   1,
   9,
   11,
   7,
   2,
   5,
   8,
   10,
   1,
   12,
   7,
   12,
   6,
   6,
   11,
   8,
   3,
   12,
   7,
   9,
   11,
   9,
   5,
   5],
  [1,
   6,
   8,
   4,
   1,
   9,
   11,
   7,
   2,
   5,
   8,
   10,
   1,
   12,
   7,
   12,
   6,
   6,
   11,
   8,
   3,
   12,
   7,
   9,
   11,
   9,
   5,
   0]],
 [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1],
 True)

In [18]:
#We create a df to store the data properly
data = pd.DataFrame(data={'input_ids':input_ids, 'attention_mask':attention_mask, 'token_type_ids':token_type_ids, 'train':train})
data.head()

,input_ids,attention_mask,token_type_ids,train
0,"[[1, 6, 8, 4, 1, 9, 11, 7, 2, 5, 8, 10, 1, 12,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",True
1,"[[1, 6, 8, 4, 1, 9, 11, 7, 2, 5, 8, 10, 1, 12,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",True
2,"[[1, 6, 8, 4, 1, 9, 11, 7, 2, 5, 8, 10, 1, 12,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",True
3,"[[1, 6, 8, 4, 1, 9, 11, 7, 2, 5, 8, 10, 1, 12,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",True
4,"[[1, 6, 8, 4, 1, 9, 11, 7, 2, 5, 8, 10, 1, 12,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",True


## DatasetDict creation

In [20]:
#We turn our datasets into DatasetDicts
train = Dataset.from_pandas(data.loc[data.train == True].reset_index(drop=True)) 
test = Dataset.from_pandas(data.loc[data.train == False].reset_index(drop=True))

ds = DatasetDict()

ds['train'] = train
ds['validation'] = test

ds

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'token_type_ids', 'train'],
        num_rows: 36
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'token_type_ids', 'train'],
        num_rows: 0
    })
})

# Model training

In [52]:
#Define training arguments
training_args = TrainingArguments("test-rm",
                                  per_device_train_batch_size=1,
                                  per_device_eval_batch_size=24,
                                  num_train_epochs=1,
                                  learning_rate = 3e-05,
                                  evaluation_strategy="epoch",
                                  save_steps = 1500,
                                  )

In [53]:
#We redefine the trainer in order to get the custom loss that the RM needs
class CustomTrainer(Trainer):
        def compute_loss(self, model, inputs, return_outputs=False):
            out_right, out_wrong = model(
                input_ids = inputs['input_ids'],
                attention_mask = inputs['attention_mask'],
                token_type_ids = inputs['token_type_ids'],
                train = inputs['train']
            )
            #The idea behind the equations is better explained in the following article: https://medium.com/towards-generative-ai/reward-model-training-2209d1befb5f
            diff = out_right-out_wrong
            print(diff)
            sig_out = nn.Sigmoid(diff)
            loss = -np.log(sig_out)
            return (loss, diff) if return_outputs else loss

In [54]:
#We create the trainer
trainer = CustomTrainer(
    rm,
    training_args,
    train_dataset=ds['train'])

In [55]:
#Finally, we train the model
trainer.train()

tensor([[[ 9,  5,  6,  9, 11,  1,  2,  6,  8, 12,  0,  6,  7, 10, 12,  9, 11,
          11,  3,  0,  3,  1,  7, 30, 30, 30, 30, 30],
         [ 9,  5,  6,  9, 11,  1,  2,  6,  8, 12,  0,  6,  7, 10, 12,  9, 11,
          11,  3,  0,  3,  1,  4, 30, 30, 30, 30, 30]]], device='cuda:0')


ValueError: not enough values to unpack (expected 2, got 1)